## Streamlit Google Colab

In [47]:
!pip install pyngrok
!pip install streamlit
# then restart runtime

In [48]:
%%writefile app.py
import matplotlib.pyplot as plt
import numpy as np
import streamlit as st
from matplotlib.patches import Circle


@st.cache(allow_output_mutation=True)
def magnetic_field_e(q, r0, x, y) -> tuple:
    """Retorna el vector de campo eléctrico E=(Ex,Ey) de una carga q en r0"""
    den = np.hypot(x - r0[0], y - r0[1]) ** 3
    return q * (x - r0[0]) / den, q * (y - r0[1]) / den


@st.cache(allow_output_mutation=True)
def magnetic_field_vector(n: int, axis_x: int, axis_y: int) -> dict:
    """Cálculo del campo electromagnetico según la cantidad de dipolos.

    Args:
        n (int): Número de cargas para definir el dipolo

    Returns:
        dict: x,y,charges,Ex,Ey
    """
    # # puntos de los ejes x e y.
    nx, ny = 128, 128
    x = np.linspace(-axis_x, axis_x, nx)
    y = np.linspace(-axis_y, axis_y, ny)
    X, Y = np.meshgrid(x, y)
    # # Crear un multipolo con nq cargas
    # count = número de q. En ese caso es 1 dipolo
    count = n
    nq = 2 ** int(count)
    charges = [
        (i % 2 * 2 - 1, (np.cos(2 * np.pi * i / nq), np.sin(2 * np.pi * i / nq)))
        for i in range(nq)
    ]

    # # Vector de campo eléctrico como componentes separados (Ex,Ey)
    Ex, Ey = np.zeros((ny, nx)), np.zeros((ny, nx))
    for charge in charges:
        ex, ey = magnetic_field_e(*charge, x=X, y=Y)
        Ex += ex
        Ey += ey

    return {"x": x, "y": y, "charges": charges, "Ex": Ex, "Ey": Ey}


def main():
    """Generación de la webapp con streamlit"""
    # Definir título
    st.title("Magnetic fields")
    # Entrada de datos en el sidebar.
    st.sidebar.header("Inputs:")
    # min: 1, max: 5, default: 1
    n = int(st.sidebar.slider("Charges", 1, 10, 1))
    # SelectBox en sidebar
    min_max_scale = int(
        st.sidebar.selectbox("min,max value in graph", list(range(11, 21)))
    )
    # Graficar usando matplotlib
    fig = plt.figure()
    ax = fig.add_subplot(111)
    # Se realiza el cálculod el vector de campo magnético.
    res = magnetic_field_vector(n, min_max_scale, min_max_scale)
    # Muestra la cantidad de polos que se definen en la entrada de datos.
    st.header(f"{2*n} Polos")
    # Dibujar las líneas de flujo con mapa de colores y estilos apropiados.
    color = 2 * np.log(np.hypot(res["Ex"], res["Ey"]))
    ax.streamplot(
        res["x"],
        res["y"],
        res["Ex"],
        res["Ey"],
        color=color,
        linewidth=1,
        cmap=plt.cm.inferno,
        density=2,
        arrowstyle="->",
        arrowsize=1.5,
    )
    # Agregar circulos para las cargas.
    charge_colors = {True: "#aa0000", False: "#0000aa"}

    for q, pos in res["charges"]:
        ax.add_artist(Circle(pos, 0.05, color=charge_colors[q > 0]))

    # Graficar
    ax.set_xlabel("$x$")
    ax.set_ylabel("$y$")
    ax.set_xlim(-min_max_scale, min_max_scale)
    ax.set_ylim(-min_max_scale, min_max_scale)
    ax.set_aspect("equal")
    # plt.show()
    st.pyplot(fig)


if __name__ == "__main__":
    main()


###



Overwriting app.py


In [64]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

get_ipython().system_raw('tensorboard --logdir /content/trainingdata/objectdetection/ckpt_output/trainingImatges/ --host 0.0.0.0 --port 6006 &')

get_ipython().system_raw('./ngrok http 6006 &')

! curl -s http://localhost:8050/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2020-10-26 16:03:22--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.236.132.162, 3.225.89.236, 34.206.161.174, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.236.132.162|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.4’

ngrok-stable-linux- 100%[===================>]  13.13M  41.9MB/s    in 0.3s    

2020-10-26 16:03:22 (41.9 MB/s) - ‘ngrok-stable-linux-amd64.zip.4’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/usr/lib/python3.6/json/__init__.py", line 299, in load
    parse_constant=parse_constant, object_pairs_hook=object_pairs_hook, **kw)
  File "/usr/lib/python3.6/json/__init__.py", line 354, in loads
    

In [67]:
from pyngrok import ngrok
!pkill -9 streamlit && pkill -9 ngrok
#!ngrok authtoken 1imd4alFueHqqwPwn68g5kN3Y5i_X8U92xzdmE3oBpZMNYB1
!streamlit run app.py&>/dev/null&
!pgrep streamlit
public_url = ngrok.connect(port='8501')
public_url

2378


<NgrokTunnel: "http://5ea05a6af6a0.ngrok.io" -> "http://localhost:80">

In [ ]:
!top

In [66]:
!kill -9 2329

In [60]:
ngrok.kill()